In [ ]:
!sudo apt-get update
!sudo apt-get install -y gcc g++
!sudo apt-get install -y clang
!pip install ipywidgets

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
gcc is already the newest version (4:11.2.0-1ubuntu1).
0 upgraded, 0 newly installe

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import time

def compile_and_run(change=None):
    compiler = compiler_dropdown.value
    compiler_args = compiler_args_input.value.split()
    output_widget.clear_output()
    disassembly_output.clear_output()
    files_to_compile = []
    for file_name, editor in code_editors.items():
        with open(file_name, 'w') as file:
            file.write(editor.value)
        # Only consider .cpp files for compilation
        if file_name.endswith('.cpp'):
            files_to_compile.append(file_name)
    compile_result = subprocess.run([compiler] + compiler_args + files_to_compile + ['-o', 'main'], capture_output=True)
    if compile_result.returncode != 0:
        result = compile_result.stderr.decode('utf-8')
        with output_widget:
            print(result)
        return
    run_result = subprocess.run(['./main'], capture_output=True)
    result = run_result.stdout.decode('utf-8')
    disassembly_result = subprocess.run(['objdump', '-d', 'main'], capture_output=True)
    disassembly = disassembly_result.stdout.decode('utf-8')
    with output_widget:
        print(result)
    with disassembly_output:
        print(disassembly)

# Function to create a new code editor
def create_code_editor(file_name, initial_code=''):
    code_editor = widgets.Textarea(
        value=initial_code,
        placeholder=f'Type code for {file_name}',
        description=f'{file_name}:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='200px')
    )
    code_editor.observe(compile_and_run, names='value')
    code_editors[file_name] = code_editor
    editor_container.children += (code_editor,)
    update_remove_file_dropdown()

# Create a dropdown to select the compiler
compiler_dropdown = widgets.Dropdown(
    options=['g++', 'clang++'],
    value='g++',
    description='Compiler:',
    disabled=False,
)
compiler_dropdown.observe(compile_and_run, names='value')

# Create a text input for the new file name
file_name_input = widgets.Text(
    value='main.cpp',
    placeholder='Enter file name',
    description='New File Name:',
    disabled=False
)

# Create a text input for compiler arguments
compiler_args_input = widgets.Text(
    value='',
    placeholder='Enter compiler arguments',
    description='Compiler Args:',
    disabled=False
)
compiler_args_input.observe(compile_and_run, names='value')

# Create a dropdown for selecting a file to remove
remove_file_dropdown = widgets.Dropdown(
    description='Remove File:',
    disabled=False,
)

# Create a container for code editors
code_editors = {}
editor_container = widgets.VBox([])

# Create an output widget to display the disassembly
disassembly_output = widgets.Output()

# Create a button to add a new code editor
add_editor_button = widgets.Button(
    description='Add File',
    disabled=False,
    button_style='info',
    tooltip='Click to add a new code editor',
    icon='plus'
)

# Create a button to remove a selected file
remove_editor_button = widgets.Button(
    description='Remove Selected File',
    disabled=False,
    button_style='danger',
    tooltip='Click to remove the selected file',
    icon='minus'
)

# Create an output widget to display the results
output_widget = widgets.Output()

# Define the add editor action
def add_editor(b):
    file_name = file_name_input.value
    if file_name and file_name not in code_editors:
        create_code_editor(file_name)

# Define the remove editor action
def remove_editor(b):
    file_name = remove_file_dropdown.value
    if file_name:
        remove_code_editor(file_name)
        compile_and_run()

# Function to remove a code editor
def remove_code_editor(file_name):
    if file_name in code_editors:
        del code_editors[file_name]
        editor_container.children = [editor for editor in editor_container.children if editor.description != f'{file_name}:']
        update_remove_file_dropdown()

# Function to update the remove file dropdown options
def update_remove_file_dropdown():
    remove_file_dropdown.options = list(code_editors.keys())

# Assign actions to buttons
add_editor_button.on_click(add_editor)
remove_editor_button.on_click(remove_editor)

# Create initial code editors
create_code_editor('main.cpp', '#include <iostream>\n\nint main() {\n    std::cout << "Hello, World!" << std::endl;\n    return 0;\n}')

# Display the widgets
display(compiler_dropdown, file_name_input, add_editor_button, remove_file_dropdown, remove_editor_button, compiler_args_input, editor_container, output_widget, disassembly_output)

Dropdown(description='Compiler:', options=('g++', 'clang++'), value='g++')

Text(value='main.cpp', description='New File Name:', placeholder='Enter file name')

Button(button_style='info', description='Add File', icon='plus', style=ButtonStyle(), tooltip='Click to add a …

Dropdown(description='Remove File:', options=('main.cpp',), value='main.cpp')

Button(button_style='danger', description='Remove Selected File', icon='minus', style=ButtonStyle(), tooltip='…

Text(value='', description='Compiler Args:', placeholder='Enter compiler arguments')

Output()

Output()